In [5]:
import org.apache.spark.sql.functions.{initcap,col}

In [ ]:
val spark_home = "/Users/fanzhenxin/bigData/spark-2.4.4-bin-hadoop2.7"
val data_path = spark_home+"/data"
val df_path = data_path+"/retail-data/by-day/2010-12-01.csv"
val df = spark.read.format("csv")
    .option("header","true")
    .option("inferSchema","true")
    .load(df_path)
df.createOrReplaceTempView("dfTable")

In [6]:
df.select(initcap(col("Description"))).show(2,false)

+----------------------------------+
|initcap(Description)              |
+----------------------------------+
|White Hanging Heart T-light Holder|
|White Metal Lantern               |
+----------------------------------+
only showing top 2 rows



In [8]:
import org.apache.spark.sql.functions.{lower,upper}

df.select(col("Description"),
         lower(col("Description")),
         upper(lower(col("Description")))).show(2)

+--------------------+--------------------+-------------------------+
|         Description|  lower(Description)|upper(lower(Description))|
+--------------------+--------------------+-------------------------+
|WHITE HANGING HEA...|white hanging hea...|     WHITE HANGING HEA...|
| WHITE METAL LANTERN| white metal lantern|      WHITE METAL LANTERN|
+--------------------+--------------------+-------------------------+
only showing top 2 rows



In [12]:
import org.apache.spark.sql.functions.{lit,ltrim,rtrim,rpad,lpad,trim}

df.select(
    ltrim(lit("    HELLO    ")).as("ltrim"),
    rtrim(lit("    HELLO    ")).as("rtrim"),
    trim(lit("    HELLO    ")).as("trim"),
    lpad(lit("HELLO"), 1, " ").as("lp"),
    rpad(lit("HELLO"), 10, " ").as("rp")).show(2)

+---------+---------+-----+---+----------+
|    ltrim|    rtrim| trim| lp|        rp|
+---------+---------+-----+---+----------+
|HELLO    |    HELLO|HELLO|  H|HELLO     |
|HELLO    |    HELLO|HELLO|  H|HELLO     |
+---------+---------+-----+---+----------+
only showing top 2 rows



In [19]:
import org.apache.spark.sql.functions.regexp_replace

val simpleColors = Seq("black","white","red","green","blue")

val regexString = simpleColors.map(_.toUpperCase).mkString("|")
val reg = "BLACK|WHITE|RED|GREEN|BLUE"

print(regexString==reg)
df.select(
regexp_replace(col("Description"),reg,"COLOR").alias("color_clean"),
    col("Description")
).show(2,false)

true+----------------------------------+----------------------------------+
|color_clean                       |Description                       |
+----------------------------------+----------------------------------+
|COLOR HANGING HEART T-LIGHT HOLDER|WHITE HANGING HEART T-LIGHT HOLDER|
|COLOR METAL LANTERN               |WHITE METAL LANTERN               |
+----------------------------------+----------------------------------+
only showing top 2 rows



simpleColors = List(black, white, red, green, blue)
regexString = BLACK|WHITE|RED|GREEN|BLUE
reg = BLACK|WHITE|RED|GREEN|BLUE


BLACK|WHITE|RED|GREEN|BLUE

In [22]:
import org.apache.spark.sql.functions.translate

df.select(translate(col("Description"),"LEET","4337"),
         col("Description"))
.show(2,false)

+----------------------------------+----------------------------------+
|translate(Description, LEET, 4337)|Description                       |
+----------------------------------+----------------------------------+
|WHI73 HANGING H3AR7 7-4IGH7 HO4D3R|WHITE HANGING HEART T-LIGHT HOLDER|
|WHI73 M37A4 4AN73RN               |WHITE METAL LANTERN               |
+----------------------------------+----------------------------------+
only showing top 2 rows



In [23]:
import org.apache.spark.sql.functions.regexp_extract

val regexString=
simpleColors.map(_.toUpperCase).mkString("(","|",")")
df.select(
regexp_extract(col("Description"),regexString,1).alias("color_clean"),
    col("Description")
).show(2)

+-----------+--------------------+
|color_clean|         Description|
+-----------+--------------------+
|      WHITE|WHITE HANGING HEA...|
|      WHITE| WHITE METAL LANTERN|
+-----------+--------------------+
only showing top 2 rows



regexString = (BLACK|WHITE|RED|GREEN|BLUE)


(BLACK|WHITE|RED|GREEN|BLUE)

In [24]:
val containsBlack = col("Description").contains("BLACK")
val containsWhite = col("DESCRIPTION").contains("WHITE")

df.withColumn("hasSimpleColor",
             containsBlack.or(containsWhite))
.where("hasSimpleColor")
.select("Description")
.show(3,false)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



containsBlack = contains(Description, BLACK)
containsWhite = contains(DESCRIPTION, WHITE)


contains(DESCRIPTION, WHITE)

In [25]:
df.select("Description").show(3,false)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|CREAM CUPID HEARTS COAT HANGER    |
+----------------------------------+
only showing top 3 rows



In [27]:
import org.apache.spark.sql.functions.expr
val simpleColors = Seq("black","white","red","green","blue")

val selectedColumns = simpleColors.map(color=>{
    col("Description").contains(color.toUpperCase).alias(s"is_$color")
}):+expr("*")

df.select(selectedColumns:_*).where(col("is_white").or(col("is_red")))
.select("Description").show(3,false)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



simpleColors = List(black, white, red, green, blue)
selectedColumns = List(contains(Description, BLACK) AS `is_black`, contains(Description, WHITE) AS `is_white`, contains(Description, RED) AS `is_red`, contains(Description, GREEN) AS `is_green`, contains(Description, BLUE) AS `is_blue`, unresolvedstar())


List(contains(Description, BLACK) AS `is_black`, contains(Description, WHITE) AS `is_white`, contains(Description, RED) AS `is_red`, contains(Description, GREEN) AS `is_green`, contains(Description, BLUE) AS `is_blue`, unresolvedstar())